# Historical Crypto Price Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_style('darkgrid')
sns.set_context('poster', font_scale=1)

from time import sleep, time
from datetime import timedelta, datetime, date, timezone
import requests
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 60

import coinmarketcap
import json

from sklearn.preprocessing import normalize

In [68]:
testing = max(etc.date)

In [80]:
datetime(testing.year, testing.month, testing.day)

datetime.datetime(2018, 2, 20, 0, 0)

## CryptoCompare
https://www.cryptocompare.com/api/

In [2]:
def get_all_coin_symbols():
    a = requests.get('https://www.cryptocompare.com/api/data/coinlist/')
    a = a.json()['Data']
    coins = [i for i in a]
    print (len(coins), 'coins on the market, today!')
    return coins

In [3]:
coins = get_all_coin_symbols()

2189 coins on the market, today!


## CoinMarketCap

In [24]:
def get_cmc_data():
    market = coinmarketcap.Market()
    cmc = market.ticker()
    df = pd.DataFrame(cmc)
    df = df[['rank', 'name', 'symbol', 'market_cap_usd', 'price_usd', 'price_btc', 'available_supply', 'max_supply',
             'percent_change_1h', 'percent_change_24h', 'percent_change_7d', '24h_volume_usd']]
    for i in df.columns[3:]:
        df[i] = df[i].astype(float)
    return df

cmc_top100 = get_cmc_data()
top100_sym = list(cmc_top100.symbol)

In [25]:
cmc_top100.sort_values('percent_change_1h', ascending=False).head(10)

,rank,name,symbol,market_cap_usd,price_usd,price_btc,available_supply,max_supply,percent_change_1h,percent_change_24h,percent_change_7d,24h_volume_usd
38,39,ZClassic,ZCL,6.062116e+08,180.904000,1.543780e-02,3.351013e+06,21000000.0,5.08,-6.28,25.82,25605900.0
83,84,Pillar,PLR,2.570426e+08,1.130430,9.647000e-05,2.273848e+08,NaN,4.82,0.86,-2.91,522794.0
20,21,OmiseGO,OMG,2.024555e+09,19.840300,1.693110e-03,1.020426e+08,NaN,2.58,11.64,53.82,94166700.0
74,75,ReddCoin,RDD,2.999511e+08,0.010429,8.900000e-07,2.876180e+10,NaN,2.22,36.07,59.64,103938000.0
50,51,Ark,ARK,4.548513e+08,4.514100,3.852200e-04,1.007623e+08,NaN,2.20,-4.14,10.97,4726660.0
28,29,Populous,PPT,9.724473e+08,26.279500,2.242620e-03,3.700403e+07,53252246.0,1.77,-6.82,-1.01,4240410.0
88,89,Vertcoin,VTC,2.195029e+08,5.102550,4.354400e-04,4.301828e+07,84000000.0,1.57,10.24,62.20,14004300.0
14,15,TRON,TRX,3.238624e+09,0.049258,4.200000e-06,6.574819e+10,NaN,1.52,-4.65,13.49,196843000.0
64,65,Byteball Bytes,GBYTE,3.454945e+08,535.466000,4.569500e-02,6.452220e+05,NaN,1.42,-12.97,-6.36,2143580.0
95,96,TenX,PAY,2.023187e+08,1.933080,1.649600e-04,1.046613e+08,NaN,1.23,-7.42,5.74,3541310.0


## CryptoCompare
HistoHour API

**Add function to read in historical prices csv and then udpate the df since last price timestamp**

In [5]:
def get_hourly_close(fsym, tsym):
    if fsym == 'BTC':
        tsym = 'USD'
    to_timestamp = int(time())  # current time
    df = pd.DataFrame()
    for i in range(10):
        r = requests.get('https://min-api.cryptocompare.com/data/histohour?fsym=' + fsym + '&tsym=' + tsym + '&limit=2000&aggregate=1&toTs=' + str(to_timestamp))
        assert r.status_code == 200
        assert r.json()['Response'] == 'Success'
        sleep(0.25)

        data = r.json()['Data']
        if len(data) != 0:
            data_list = [(datetime.utcfromtimestamp(j['time']), j['close']) for j in data]
            df = df.append(pd.DataFrame(data_list))
        else:
            break
        
        to_timestamp -= 3600*2001
    
    df.columns = ['date', fsym]
    df.date = pd.to_datetime(df.date)
    df.sort_values('date', inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df

In [6]:
def get_top_coins_close(coin_list, coins):
    top_coins = pd.DataFrame()
    for first, coin_sym in enumerate(coin_list):
        if coin_sym in coins:
            print (coin_sym)
            df = get_hourly_close(coin_sym, 'BTC')
            if first == 0:
                top_coins = df
            else:
                top_coins = top_coins.merge(df, on='date', how='outer')
        else:
            print (coin_sym, '--- not in coins')
    top_coins.sort_values('date', inplace=True)
    top_coins.reset_index(inplace=True, drop=True)
    return top_coins

In [ ]:
## UPDATE THIS FUNCTION
def update_histo_price_df(filepath):
    saved_df = pd.read_csv(filepath)
    saved_df['date'] = pd.to_datetime(saved_df.date)
    last_date = max(saved_df.date)

In [152]:
%time histo_top100 = get_top_coins_close(top100_sym, coins)

BTC
ETH
XRP
BCH
ADA
LTC
XLM
NEO
EOS
XEM
IOTA --- not in coins
DASH
XMR
TRX
LSK
VEN
ETC
PPT
QTUM
USDT
XRB
ICX
BTG
OMG
ZEC
STEEM
BNB
STRAT
XVG
SC
BCN
BTS
DGD
MKR
WTC
ZRX
VERI
KCS
SNT
WAVES
AE
REP
RHOC
DCR
DOGE
ARDR
KNC
HSR
KMD
GAS
ZIL
DRGN
DCN
ARK
LRC
ETN
BAT
DGB
GBYTE
ELF
PIVX
QASH
GNT
ZCL
BTM
NAS
DENT
ETHOS --- not in coins
IOST
PLR
CND
CNX
GXS
SALT
AION
FCT
R
SYS
BTX
POWR
AGI
FUN
KIN
MONA
XZC
SMART
NXT
ENG
RDD
REQ
MAID
NXS
PART
IGNIS
PAY
WAX
ICN
BNT
GAME
GNO
CPU times: user 1min 19s, sys: 1.53 s, total: 1min 21s
Wall time: 10min 59s


In [11]:
histo_top100 = pd.read_csv('../exported_csvs/historical-prices.csv', index_col=0)

In [12]:
print (histo_top100.shape)
histo_top100.head(10)

(20010, 98)


,date,BTC,ETH,XRP,BCH,LTC,ADA,NEO,XLM,EOS,DASH,XMR,XEM,ETC,VEN,TRX,LSK,QTUM,BTG,USDT,OMG,ICX,ZEC,XVG,BCN,STEEM,BNB,PPT,STRAT,SC,RHOC,DOGE,WAVES,SNT,MKR,BTS,AE,R,WTC,ZCL,DCR,ZRX,VERI,REP,DGD,ARDR,HSR,ETN,KMD,KCS,GAS,ARK,DGB,ZIL,BAT,DRGN,SYS,LRC,ELF,POLY,BTM,CNX,MONA,GBYTE,QASH,GNT,AION,PIVX,NAS,KNC,IOST,DCN,RDD,BTX,GXS,FCT,XZC,LINK,POWR,DENT,FUN,NXT,PLR,KIN,SALT,CND,PART,NEBL,ENG,BNT,RDN,VTC,MAID,BLOCK,EMC,REQ,SMART,PAY
0,2015-11-09 22:00:00,379.31,0.002672,0.000012,0.0,0.008881,0.0,0.0,0.000005,0.0,0.006727,0.001242,3.700000e-07,0.0,0.000146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.000000e-08,0.0,0.0,0.0,0.0,9.000000e-08,0.0,3.500000e-07,0.0,0.0,0.0,0.000008,0.0,0.0,0.0,0.0,0.00029,0.0,0.0,0.005155,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.700000e-07,0.0,0.0,0.0,0.000001,0.0,0.0,1.000000e-08,0.000086,0.0,0.000204,0.0,0.0,0.0,0.0,0.0,5.000000e-08,0.000180,0.0,0.0,3.000000e-08,5.500000e-07,0.0,0.000340,0.0,0.0,0.0,0.0,0.0,0.000018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000008,0.000078,0.000035,0.000076,0.0,0.0,0.0,0.0
1,2015-11-09 23:00:00,380.33,0.002636,0.000012,0.0,0.008894,0.0,0.0,0.000005,0.0,0.006772,0.001231,4.000000e-07,0.0,0.000146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.000000e-08,0.0,0.0,0.0,0.0,9.000000e-08,0.0,3.600000e-07,0.0,0.0,0.0,0.000008,0.0,0.0,0.0,0.0,0.00029,0.0,0.0,0.005155,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.700000e-07,0.0,0.0,0.0,0.000001,0.0,0.0,1.000000e-08,0.000086,0.0,0.000211,0.0,0.0,0.0,0.0,0.0,5.000000e-08,0.000180,0.0,0.0,3.000000e-08,5.500000e-07,0.0,0.000340,0.0,0.0,0.0,0.0,0.0,0.000017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000008,0.000078,0.000035,0.000076,0.0,0.0,0.0,0.0
2,2015-11-10 00:00:00,375.30,0.002621,0.000012,0.0,0.008944,0.0,0.0,0.000005,0.0,0.006602,0.001236,3.900000e-07,0.0,0.000146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.000000e-08,0.0,0.0,0.0,0.0,8.000000e-08,0.0,3.600000e-07,0.0,0.0,0.0,0.000008,0.0,0.0,0.0,0.0,0.00029,0.0,0.0,0.005152,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.100000e-07,0.0,0.0,0.0,0.000001,0.0,0.0,1.000000e-08,0.000086,0.0,0.000211,0.0,0.0,0.0,0.0,0.0,5.000000e-08,0.000180,0.0,0.0,4.000000e-08,5.500000e-07,0.0,0.000340,0.0,0.0,0.0,0.0,0.0,0.000017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000008,0.000078,0.000035,0.000078,0.0,0.0,0.0,0.0
3,2015-11-10 01:00:00,374.28,0.002635,0.000012,0.0,0.008851,0.0,0.0,0.000005,0.0,0.006499,0.001231,4.000000e-07,0.0,0.000146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.000000e-08,0.0,0.0,0.0,0.0,8.000000e-08,0.0,3.600000e-07,0.0,0.0,0.0,0.000008,0.0,0.0,0.0,0.0,0.00029,0.0,0.0,0.005080,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.600000e-07,0.0,0.0,0.0,0.000001,0.0,0.0,1.000000e-08,0.000086,0.0,0.000210,0.0,0.0,0.0,0.0,0.0,5.000000e-08,0.000180,0.0,0.0,3.000000e-08,5.500000e-07,0.0,0.000354,0.0,0.0,0.0,0.0,0.0,0.000018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000008,0.000079,0.000035,0.000078,0.0,0.0,0.0,0.0
4,2015-11-10 02:00:00,376.08,0.002641,0.000012,0.0,0.008929,0.0,0.0,0.000005,0.0,0.006578,0.001233,3.900000e-07,0.0,0.000146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.000000e-08,0.0,0.0,0.0,0.0,8.000000e-08,0.0,3.600000e-07,0.0,0.0,0.0,0.000008,0.0,0.0,0.0,0.0,0.00029,0.0,0.0,0.005132,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.500000e-07,0.0,0.0,0.0,0.000001,0.0,0.0,1.000000e-08,0.000086,0.0,0.000208,0.0,0.0,0.0,0.0,0.0,5.000000e-08,0.000180,0.0,0.0,3.000000e-08,5.500000e-07,0.0,0.000359,0.0,0.0,0.0,0.0,0.0,0.000018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000008,0.000079,0.000035,0.000078,0.0,0.0,0.0,0.0
5,2015-11-10 03:00:00,376.58,0.002641,0.000012,0.0,0.008938,0.0,0.0,0.000005,0.0,0.006654,0.001234,4.000000e-07,0.0,0.000146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.000000e-08,0.0,0.0,0.0,0.0,8.000000e-08,0.0,3.500000e-07,0.0,0.0,0.0,0.000008,0.0,0.0,0.0,0.0,0.00029,0.0,0.0,0.005132,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.500000e-07,0.0,0.0,0.0,0.000001,0.0,0.0,1.000000e-08,0.000086,0.0,0.000202,0.0,0.0,0.0,0.0,0.0,5.000000e-08,0.000231,0.0,0.0,3.000000e-08,5.500000e-07,0.0,0.000359,0.0,0.0,0.0,0.0,0.0,0.000017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000008,0.000079,0.000035,0.000078,0.0,0

In [13]:
histo_top100.describe(include='all')

,date,BTC,ETH,XRP,BCH,LTC,ADA,NEO,XLM,EOS,DASH,XMR,XEM,ETC,VEN,TRX,LSK,QTUM,BTG,USDT,OMG,ICX,ZEC,XVG,BCN,STEEM,BNB,PPT,STRAT,SC,RHOC,DOGE,WAVES,SNT,MKR,BTS,AE,R,WTC,ZCL,DCR,ZRX,VERI,REP,DGD,ARDR,HSR,ETN,KMD,KCS,GAS,ARK,DGB,ZIL,BAT,DRGN,SYS,LRC,ELF,POLY,BTM,CNX,MONA,GBYTE,QASH,GNT,AION,PIVX,NAS,KNC,IOST,DCN,RDD,BTX,GXS,FCT,XZC,LINK,POWR,DENT,FUN,NXT,PLR,KIN,SALT,CND,PART,NEBL,ENG,BNT,RDN,VTC,MAID,BLOCK,EMC,REQ,SMART,PAY
count,20010,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,2.001000e+04,2.001000e+04,2.001000e+04,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,2.001000e+04,2.001000e+04,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000,20010.000000
unique,20010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2016-08-06 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2698.321697,0.037411,0.000035,0.031025,0.009364,0.000004,0.001918,7.671482e-06,0.000141,0.035628,0.011932,2.541375e-05,0.001958,0.000436,4.602774e-07,0.000540,0.000932,0.002905,0.000120,0.000372,0.000053,0.091099,9.970940e-07,2.295467e-07,0.000510,0.000119,0.000364,0.000500,1.159900e-06,0.000010,4.503138e-07,0.000472,0.000004,0.038063,0.000018,0.000065,0.000012,0.000251,0.001381,0.004673,0.000016,0.092920,0.009853,0.015755,0.000024,0.000485,9.386417e-07,0.000163,0.000050,0.000876,0.000145,1.955392e-06,9.626787e-08,0.000014,0.000015,2.362944e-05,0.000008,0.000008,3.177275e-06,0.000175,0.000078,0.000179,0.045670,0.000011,0.000042,0.000024,0.000284,4.801870e-05,9.204263e-04,1.989830e-07,1.289560e-08,2.032689e-07,4.401205e-04,0.000037,0.003689,0.001955,0.000008,0.000009,1.899350e-07,2.040986e-04,0.000023,0.000010,29.553036,0.000108,0.000002,0.000446,0.000161,0.000029,2.925534e-04,7.550000e-06,0.000160,0.000118,0.001257,0.000358,0.000004,0.000005,0.000106
std,NaN,3852.966302,0.031845,0.000039,0.058509,0.004377,0.000013,0.003315,1.052504e-05,0.000293,0.027104,0.008507,2.952121

#### Price History Correlation

In [14]:
corr = histo_top100.corr()
corr = corr.merge(cmc_top100[['symbol', 'name']], left_index=True, right_on='symbol')
corr = corr.set_index('name').reset_index().set_index('symbol')

In [20]:
corr.sort_values('BTC', ascending=False)[['name', 'BTC']][0:10]

,name,BTC
symbol,,
BTC,Bitcoin,1.000000
MONA,MonaCoin,0.882437
POWR,Power Ledger,0.840317
ETN,Electroneum,0.835022
BCH,Bitcoin Cash,0.833442
VTC,Vertcoin,0.826087
SALT,SALT,0.818050
QASH,QASH,0.801245
PPT,Populous,0.782864
